In [1]:
import spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import re

load_dotenv('../secrets/.env')
%run useful_functions.ipynb

In [2]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

In [3]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [7]:
def get_request(q):
    return(spotify.search(q, limit=1, offset=0, type='track', market="GB")['tracks']['items'][0])
    
def get_track_info(artistName, trackName):
    original_artistName = artistName
    original_trackName = trackName
    i = 1
    while i<=3:
        try:
            q = "artist:" + artistName + " track:" + trackName
            data = get_request(q)
            break
        except:
            if i == 1:
                artistName = re.sub("(\[.*\])|(\(.*\))|(\*.*\*)",  '',    artistName).strip()  
                trackName = re.sub("(\[.*\])|(\(.*\))|(\*.*\*)",  '',    trackName).strip()
            if i == 2:
                artistName = re.sub("[',’]",  '',    artistName).strip()  
                trackName = re.sub("[',’]",  '',    trackName).strip()
            i += 1
            
    if i == 4:
        print('Error with: ', artistName, ":", trackName)
        return(pd.DataFrame(),pd.DataFrame(),pd.DataFrame())

    artist_data = data['artists']
    album_data = data['album']
    album_artists = album_data['artists']
    data.pop('album',None)
    data.pop('artists',None)

    artist_data_df = pd.concat([pd.DataFrame(artist_data),pd.DataFrame(album_artists)])
    artist_data_df.rename(columns = {'external_urls': "Artist External URL's",
                        'href': "Artist href",
                        'id': "Artist ID",
                        'name': "Artist Name",
                        'type': "Artist Type",
                        'uri': "Artist URI"}, inplace=True)
    artist_data_df.drop_duplicates(subset='Artist ID', keep='first')

    album_data['artists'] = [album_artist['id'] for album_artist in album_data['artists']]
    album_data_df = pd.DataFrame({k: [v] for k, v in album_data.items()})
    album_data_df.rename(columns={'album_type': "Album Type", 
                            'artists': "Album Artists", 
                            'external_urls': "External URL's", 
                            'href': "href", 
                            'id': "Album ID", 
                            'images': "Album Images",
                            'name': "Album Name", 
                            'release_date': "Album Release Date", 
                            'release_date_precision': "Album Release Date Precision", 
                            'total_tracks': "Album Total Tracks",
                            'type': "Album Object Type", 
                            'uri': "Album URI"}, inplace=True)
    
    data['Original Search Artist'] = original_artistName
    data['Original Search Track'] = original_trackName
    data['Cleaned Search Artist'] = artistName
    data['Cleaned Search Track'] = trackName
    data['Search Query'] = q
    data['Album ID'] = album_data['id']
    data["Artist ID's"] = [artist['id'] for artist in artist_data]
    track_data_df = pd.DataFrame({k: [v] for k, v in data.items()})
    track_data_df.rename(columns={'disc_number': 'Disc Number', 
                            'duration_ms': 'Track Duration (ms)', 
                            'explicit': 'Is Track Explicit', 
                            'external_ids': "Track External ID's",
                            'external_urls': "Track External URL's",
                            'href': 'Track href', 
                            'id': 'Track ID', 
                            'is_local': 'Is Track Local', 
                            'is_playable': 'Is Track Playable', 
                            'name': 'Track Name',
                            'popularity': 'Track Popularity', 
                            'preview_url': 'Track Preview URL', 
                            'track_number': 'Track Number', 
                            'type': 'Track Type', 
                            'uri': 'Track URI'}, inplace=True)
    
    return(track_data_df,album_data_df,artist_data_df)

In [8]:
artistName = "Loyle Carner"
trackName = "Loose Ends"
(temp_tracks,temp_albums,temp_artists) = get_track_info(artistName,trackName)


In [9]:
mask_2022 = (broadcast_details['Service Date'].apply(lambda date: date.year) == 2022)

temp = (
broadcast_details[mask_2022]
.merge(broadcast_playlists, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)
temp =  temp[temp['Show Name'] == "Radio 1's Indie Show with Jack Saunders"]

tracks = pd.DataFrame()
albums = pd.DataFrame()
artists = pd.DataFrame()

temp = temp[['Track Name', 'Artist Name']].sort_values(['Artist Name','Track Name'], ascending =  True).drop_duplicates(subset=['Artist Name','Track Name'], keep='first')
    
for i, (index, row) in enumerate(temp.iterrows()):
    print("Iteration ",i+1," of ",len(temp.index),":",row['Artist Name'],":", row['Track Name'])
    (temp_tracks,temp_albums,temp_artists) = get_track_info(row['Artist Name'],row['Track Name'])
    tracks = pd.concat([tracks,temp_tracks]).drop_duplicates(subset='Track ID', keep='first')
    albums = pd.concat([albums,temp_albums]).drop_duplicates(subset='Album ID', keep='first')
    artists = pd.concat([artists,temp_artists]).drop_duplicates(subset='Artist ID', keep='first')

Iteration  0  of  437 : Abbie Ozard : ford (drive)
Iteration  1  of  437 : Alex G : Runner
Iteration  2  of  437 : Alfie Templeman : Colour Me Blue
Iteration  3  of  437 : All Time Low : Sleepwalking
Iteration  4  of  437 : Amyl and the Sniffers : Hertz
Iteration  5  of  437 : Amyl and the Sniffers : Hertz (Glastonbury 2022)
Iteration  6  of  437 : Arcade Fire : Everything Now
Iteration  7  of  437 : Arcade Fire : Unconditional I (Lookout Kid)
Iteration  8  of  437 : Architects : Deep Fake
Iteration  9  of  437 : Architects : Tear Gas
Iteration  10  of  437 : Architects : when we were young
Iteration  11  of  437 : Arctic Monkeys : A Certain Romance
Iteration  12  of  437 : Arctic Monkeys : Body Paint
Iteration  13  of  437 : Arctic Monkeys : Brianstorm
Iteration  14  of  437 : Arctic Monkeys : Crying Lightning
Iteration  15  of  437 : Arctic Monkeys : Do I Wanna Know?
Iteration  16  of  437 : Arctic Monkeys : I Bet You Look Good On The Dancefloor
Iteration  17  of  437 : Arctic Monkey

In [10]:
tracks

,Disc Number,Track Duration (ms),Is Track Explicit,Track External ID's,Track External URL's,Track href,Track ID,Is Track Local,Is Track Playable,Track Name,...,Track Number,Track Type,Track URI,Original Search Artist,Original Search Track,Cleaned Search Artist,Cleaned Search Track,Search Query,Album ID,Artist ID's
0,1,202625,False,{'isrc': 'FRX872290005'},{'spotify': 'https://open.spotify.com/track/2l...,https://api.spotify.com/v1/tracks/2lgrgoUVMZLU...,2lgrgoUVMZLUWuH778Lydf,False,True,ford (drive),...,1,track,spotify:track:2lgrgoUVMZLUWuH778Lydf,Abbie Ozard,ford (drive),Abbie Ozard,ford (drive),artist:Abbie Ozard track:ford (drive),1V9bzaDBhIUpVW6tD1Jtz2,[03Xxe7sWXZW0NlayYQQHIn]
0,1,156772,False,{'isrc': 'GBCEL2200057'},{'spotify': 'https://open.spotify.com/track/5D...,https://api.spotify.com/v1/tracks/5DRnssBoVo8e...,5DRnssBoVo8e7uAQZkNT8O,False,True,Runner,...,2,track,spotify:track:5DRnssBoVo8e7uAQZkNT8O,Alex G,Runner,Alex G,Runner,artist:Alex G track:Runner,6TzgWk5HZItbFmMT7hH4bU,[6lcwlkAjBPSKnFBZjjZFJs]
0,1,196560,False,{'isrc': 'GBKPL2172069'},{'spotify': 'https://open.spotify.com/track/7k...,https://api.spotify.com/v1/tracks/7kdUWeOvx77r...,7kdUWeOvx77ryAireHCWgx,False,True,Colour Me Blue,...,9,track,spotify:track:7kdUWeOvx77ryAireHCWgx,Alfie Templeman,Colour Me Blue,Alfie Templeman,Colour Me Blue,artist:Alfie Templeman track:Colour Me Blue,2v64PkXSyny26cEsuEyGzr,[6QzMY3tnu0m56eKUnr4uCF]
0,1,187140,False,{'isrc': 'USAT22217358'},{'spotify': 'https://open.spotify.com/track/4S...,https://api.spotify.com/v1/tracks/4SOtTsuFYYlk...,4SOtTsuFYYlkOOvXMbroUv,False,True,Sleepwalking,...,1,track,spotify:track:4SOtTsuFYYlkOOvXMbroUv,All Time Low,Sleepwalking,All Time Low,Sleepwalking,artist:All Time Low track:Sleepwalking,4bI5LF5zauGsBIL9z8mLyl,[46gyXjRIvN1NL1eCB8GBxo]
0,1,153800,False,{'isrc': 'GBCVZ2100088'},{'spotify': 'https://open.spotify.com/track/1G...,https://api.spotify.com/v1/tracks/1GTPRD8z9c7k...,1GTPRD8z9c7kVDqwGbn52f,False,True,Hertz,...,5,track,spotify:track:1GTPRD8z9c7kVDqwGbn52f,Amyl and the Sniffers,Hertz,Amyl and the Sniffers,Hertz,artist:Amyl and the Sniffers track:Hertz,09hDOH6485A7SIg1FmpuX2,[3NqV2DJoAWsjl787bWaHW7]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,202435,True,{'isrc': 'QM6N22282249'},{'spotify': 'https://open.spotify.com/track/5z...,https://api.spotify.com/v1/tracks/5z1KeOpL9n1b...,5z1KeOpL9n1b3iFAJk91iu,False,True,Got Weird,...,1,track,spotify:track:5z1KeOpL9n1b3iFAJk91iu,dodie,Got Weird,dodie,Got Weird,artist:dodie track:Got Weird,2NBdHnQrggzuLN3FgeO9Om,[21TinSsF5ytwsfdyz5VSVS]
0,1,161695,True,{'isrc': 'GBCEL2100359'},{'spotify': 'https://open.spotify.com/track/4v...,https://api.spotify.com/v1/tracks/4v5vtgqfUy23...,4v5vtgqfUy23UF0YwxXRn4,False,True,I'm Better,...,8,track,spotify:track:4v5vtgqfUy23UF0YwxXRn4,flasher,I'm Better,flasher,Im Better,artist:flasher track:Im Better,5dDZ1oTb2lW1TXCogqbIJO,[1qPMBDzIv4WJoToZqNxb1f]
0,1,180888,False,{'isrc': 'QZNWS2264656'},{'spotify': 'https://open.spotify.com/track/0q...,https://api.spotify.com/v1/tracks/0qsYJYU4VJCi...,0qsYJYU4VJCi2SO9BBMctS,False,True,i don't know my place,...,1,track,spotify:track:0qsYJYU4VJCi2SO9BBMctS,frances mistry,I Don't Know My Place,frances mistry,I Don't Know My Place,artist:frances mistry track:I Don't Know My Place,3of8eJfT3mOhthr5x6XpFa,[3TRbyotWz0d84ZS6LaaZpG]
0,1,184153,False,{'isrc': 'GBKPL1828677'},{'spotify': 'https://open.spotify.com/track/6I...,https://api.spotify.com/v1/tracks/6IPwKM3fUUzl...,6IPwKM3fUUzlElbvKw2sKl,False,True,we fell in love in october,...,1,track,spotify:track:6IPwKM3fUUzlElbvKw2sKl,girl in red,October Passed Me By,girl in red,October Passed Me By,artist:girl in red track:October Passed Me By,7vud0sY43VTv28MbWiglDa,[3uwAm6vQy7kWPS2bciKWx9]


In [14]:
artistName = "Arctic Monkeys"
trackName = "There'd Better Be A Mirrorball"
q = "artist:" + artistName + " track:" + trackName
spotify.search(q, limit=1, offset=0, type='track', market="GB")#['tracks']['items'][0]

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3AArctic+Monkeys+track%3AThere%27d+Better+Be+A+Mirrorball&type=track&market=GB&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ln80lUS6He07XvHI8qqHH'},
       'href': 'https://api.spotify.com/v1/artists/7Ln80lUS6He07XvHI8qqHH',
       'id': '7Ln80lUS6He07XvHI8qqHH',
       'name': 'Arctic Monkeys',
       'type': 'artist',
       'uri': 'spotify:artist:7Ln80lUS6He07XvHI8qqHH'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/2GROf0WKoP5Er2M9RXVNNs'},
     'href': 'https://api.spotify.com/v1/albums/2GROf0WKoP5Er2M9RXVNNs',
     'id': '2GROf0WKoP5Er2M9RXVNNs',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b27307823ee6237208c835802663',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e0207823ee6237208c835802663',
       'wi

In [15]:
tracks

,Disc Number,Track Duration (ms),Is Track Explicit,Track External ID's,Track External URL's,Track href,Track ID,Is Track Local,Is Track Playable,Track Name,Track Popularity,Track Preview URL,Track Number,Track Type,Track URI,Search Artist,Search Track,Search Query,Album ID,Artist ID's
0,1,220826,False,{'isrc': 'USUG12106673'},{'spotify': 'https://open.spotify.com/track/4M...,https://api.spotify.com/v1/tracks/4MP5tSTc6LxY...,4MP5tSTc6LxY8xhrUOv9Ya,False,True,Shatter,55,https://p.scdn.co/mp3-preview/7eee203f2e883f8e...,7,track,spotify:track:4MP5tSTc6LxY8xhrUOv9Ya,Maggie Rogers,Shatter,artist:Maggie Rogers track:Shatter,2VeOtQQAJxR8VyvmoXqIbI,[4NZvixzsSefsNiIqXn0NDe]
0,1,235093,False,{'isrc': 'GBENL2203305'},{'spotify': 'https://open.spotify.com/track/1p...,https://api.spotify.com/v1/tracks/1pxLVdC8sjLD...,1pxLVdC8sjLDA2DF3uOEuW,False,True,Hurt You,46,https://p.scdn.co/mp3-preview/c69837e95ff3ffaa...,1,track,spotify:track:1pxLVdC8sjLDA2DF3uOEuW,Connie Constance,Hurt You,artist:Connie Constance track:Hurt You,6dbhEjyUna0Ts040kPrM8t,[4RB2kk5dmocmMiHFBlmOEt]
0,1,172506,False,{'isrc': 'GBKPL2206163'},{'spotify': 'https://open.spotify.com/track/34...,https://api.spotify.com/v1/tracks/34A1V9CbZITZ...,34A1V9CbZITZHJSfRhcbOs,False,True,Dressed to Kill,53,https://p.scdn.co/mp3-preview/5a599de4c308fb60...,1,track,spotify:track:34A1V9CbZITZHJSfRhcbOs,The Wombats,Dressed To Kill,artist:The Wombats track:Dressed To Kill,1UZAqjMhQ3pt1SXbvlnuqr,[0Ya43ZKWHTKkAbkoJJkwIB]
0,1,136510,True,{'isrc': 'QM24S2204915'},{'spotify': 'https://open.spotify.com/track/45...,https://api.spotify.com/v1/tracks/45WduJCqST7p...,45WduJCqST7pBJPvxZ6ZgU,False,True,Sand on the Beach,48,https://p.scdn.co/mp3-preview/88ba02bf53d1705d...,1,track,spotify:track:45WduJCqST7pBJPvxZ6ZgU,FIDLAR,Sand on the Beach,artist:FIDLAR track:Sand on the Beach,3vcKqp5Pque6sqEXiZvMtU,[3P6duIn7oHeiBACZfYeNud]
0,1,224687,False,{'isrc': 'GBUM72207153'},{'spotify': 'https://open.spotify.com/track/1S...,https://api.spotify.com/v1/tracks/1Sf0SsvcLIhV...,1Sf0SsvcLIhVlfKUg3L2Jd,False,True,Ghost Brigade,45,https://p.scdn.co/mp3-preview/5cd0770475ac433d...,1,track,spotify:track:1Sf0SsvcLIhVlfKUg3L2Jd,Creeper,Ghost Brigade,artist:Creeper track:Ghost Brigade,54LUC08HR95BMdbH4v7MVn,[0nV7SiEIVtPLTSJ6NwWDGj]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,229105,False,{'isrc': 'DECB82200418'},{'spotify': 'https://open.spotify.com/track/08...,https://api.spotify.com/v1/tracks/08wtb4YQBaqM...,08wtb4YQBaqM6mJhBMrkf3,False,True,Choc Mint,12,https://p.scdn.co/mp3-preview/ee7a47386704889b...,1,track,spotify:track:08wtb4YQBaqM6mJhBMrkf3,Mid City,Choc Mint,artist:Mid City track:Choc Mint,7fFIpLP2FvKPOcsXK2iw6X,[5CfLIXfiqkmqz5If0yFu5E]
0,1,182580,True,{'isrc': 'GBARL2200448'},{'spotify': 'https://open.spotify.com/track/1M...,https://api.spotify.com/v1/tracks/1Mv0nJfyxuZM...,1Mv0nJfyxuZM05O3GseVIw,False,True,Bad Influence,36,https://p.scdn.co/mp3-preview/2dcd85157dea9cf2...,4,track,spotify:track:1Mv0nJfyxuZM05O3GseVIw,Hot Milk,Bad Influence,artist:Hot Milk track:Bad Influence,1a3S2I3J4LcmAoj5ttDaoP,[1koutXdSFq2PHqtxSWj9tK]
0,1,264946,False,{'isrc': 'GBUM71505927'},{'spotify': 'https://open.spotify.com/track/53...,https://api.spotify.com/v1/tracks/53p7or9OC8PG...,53p7or9OC8PGNnGrd9gLBv,False,True,Bubblegum,50,https://p.scdn.co/mp3-preview/935bb28749ebfbb4...,3,track,spotify:track:53p7or9OC8PGNnGrd9gLBv,Mystery Jets,Bubblegum,artist:Mystery Jets track:Bubblegum,5gdWHfwu68jqPbxXxMDyL6,[09K1H1DgyIXHsMx2j7KTFX]
0,1,245640,False,{'isrc': 'GBUM70901126'},{'spotify': 'https://open.spotify.com/track/73...,https://api.spotify.com/v1/tracks/73M2Vb5MfZh8...,73M2Vb5MfZh8iGKudkMtlw,False,True,Always Like This,63,https://p.scdn.co/mp3-preview/ba2ff77b7a6c1d16...,6,track,spotify:track:73M2Vb5MfZh8iGKudkMtlw,Bombay Bicycle Club,Always Like This,artist:Bombay Bicycle Club track:Always Like This,4X0bhkSBOfcnOPyfG5Ef6a,[3pTE9iaJTkWns3mxpNQlJV]


324                    artist:Maggie Rogers track:Shatter
325                artist:Connie Constance track:Hurt You
326              artist:The Wombats track:Dressed To Kill
327                 artist:FIDLAR track:Sand on the Beach
328                    artist:Creeper track:Ghost Brigade
                              ...                        
2018                      artist:Mid City track:Choc Mint
2019                  artist:Hot Milk track:Bad Influence
2020                  artist:Mystery Jets track:Bubblegum
2021    artist:Bombay Bicycle Club track:Always Like This
2022      artist:Two Door Cinema Club track:What You Know
Name: q, Length: 724, dtype: object